<a href="https://colab.research.google.com/github/ericjenn/working-groups/blob/ericjenn-srpwg-wg1/safety-related-profile/documents/profile_opset/div.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install onnx onnxruntime


In [10]:
import onnx
import onnxruntime as ort
import numpy as np

# Define input and output tensor names
input1_name = "X"
input2_name = "Y"
div_output_name = "DivOutput"

# Create input tensor (two inputs for subtraction)
input1 = onnx.helper.make_tensor_value_info(input1_name, onnx.TensorProto.FLOAT, [None])
input2 = onnx.helper.make_tensor_value_info(input2_name, onnx.TensorProto.FLOAT, [None])

# Create output tensor (final result after log operation)
div_output = onnx.helper.make_tensor_value_info(div_output_name, onnx.TensorProto.FLOAT, [None])

# Define Subtraction node: Z = X - Y
div_node = onnx.helper.make_node("Div", [input1_name, input2_name], [div_output_name])

# Create the ONNX graph
graph = onnx.helper.make_graph(
    nodes=[div_node],
    name="DivExample",
    inputs=[input1, input2],
    outputs=[div_output]
)

# Create the ONNX model
model = onnx.helper.make_model(graph, opset_imports=[onnx.helper.make_opsetid("", 13)]) # Explicitly set opset to 13
onnx.checker.check_model(model)

# Save the model
onnx.save(model, "div.onnx")

# Load and run the model using ONNX Runtime
session = ort.InferenceSession("div.onnx")

# Case 1: 0/0
# Define input values
X_val = np.array([0.0], dtype=np.float32)
Y_val = np.array([0.0], dtype=np.float32)

# Run inference
outputs = session.run(None, {input1_name: X_val, input2_name: Y_val})

# Display results
print(f"Case 1: X={X_val}, Y={Y_val}")
print(f"Result: X/Y={outputs[0]} (Div result)")

# Case 2: 1/0
# Define input values
X_val = np.array([1.0], dtype=np.float32)
Y_val = np.array([0.0], dtype=np.float32)

# Run inference
outputs = session.run(None, {input1_name: X_val, input2_name: Y_val})

# Display results
print(f"Case 2: X={X_val}, Y={Y_val}")
print(f"Result X/Y={outputs[0]} (Div result)")


Case 1: X=[0.], Y=[0.]
Result: X/Y = [nan] (Div result)
Case 2: X=[1.], Y=[0.]
Result X/Y=[inf] (Div result)
